# Libraries

In [59]:
! pip install spacy

     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/45.9 kB ? eta -:--:--
     -------------------------- ------------- 30.7/4

In [60]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 129.6 kB/s eta 0:01:39
     --------------------------------------- 0.0/12.8 MB 129.6 kB/s eta 0:01:39
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 93.5 kB/s eta 0:02:17
     ---------------------------------------- 0.0/12.8 MB 81.9 kB/s eta 0:02:36
     --------------------------------------- 0.1/12.8 MB 151.0 kB/s eta 0:01:25
     --------------------------------------- 0.1/12.8 MB 204.8 kB/s eta 0:01:02
     --------------------------------------- 0.2/12.8 MB 247.6 kB/s eta 0:00:52
      -------------------------------------- 0.2/12.8 MB 303.5 k

In [61]:
import pandas as pd
import spacy


nlp = spacy.load("en_core_web_sm")



# Course List

In [66]:
core_courses = {
    "Programming Fundamentals": "3-1",
    "Object Oriented Programming": "3-1",
    "Data Structures & Algorithms": "3-1",
    "Discrete Structures": "3-0",
    "Operating Systems": "3-1",
    "Database Systems": "3-1",
    "Software Engineering": "3-0",
    "Computer Networks": "3-1",
    "Information Security": "3-0",
    "Final Year Project": "0-6"
}

general_education_courses = {
    "English Composition & Comprehension": "2-1",
    "Technical & Business Writing": "3-0",
    "Communication & Presentation Skills": "3-0",
    "Professional Practices": "3-0",
    "Introduction to Info. & Comm. Technologies": "0-3",
    "Pakistan Studies": "2-0",
    "Islamic Studies/ Ethics": "2-0"
}
university_elective_courses = {
    "Foreign Language": "2-0",
    "Social Service": "1-0",
    "Management Related": "3-0",
    "Social Science Related": "3-0",
    "Economy Related": "3-0"
}
math_science_foundation_courses = {
    "Calculus & Analytical Geometry": "3-0",
    "Probability & Statistics": "3-0",
    "Linear Algebra": "3-0",
    "Applied Physics": "3-0"
}
domain_courses = {
    "Compiler Construction": "3-0",
    "Comp. Organization & Assembly Language": "3-1",
    "Digital Logic Design": "3-1",
    "Design & Analysis of Algorithms": "3-0",
    "Parallel & Distributed Computing": "3-0",
    "Artificial Intelligence": "3-1",
    "Theory of Automata": "3-0"
}
support_courses = {
    "Differential Equations": "3-0",
    "Multi-variate Calculus": "3-0",
    "Graph Theory": "3-0",
    "Theory of Programming Languages": "3-0",
    "Numerical Computing": "3-0"
}
cs_elective_courses = {
    "CS Elective – 1": "3-0",
    "CS Elective – 2": "3-0",
    "CS Elective – 3": "3-0",
    "CS Elective – 4": "3-0",
    "CS Elective – 5": "3-0"
}

In [72]:
def find_closest_match(course_name):
    max_similarity = 0
    best_match = None
    credit_hours = 0  # Add a variable for credit hours

    all_courses = {**core_courses, **general_education_courses, **university_elective_courses, **math_science_foundation_courses, **domain_courses, **cs_elective_courses, **support_courses}

    for key, value in all_courses.items():
        similarity = nlp(course_name).similarity(nlp(key))
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = key
            credit_hours = value  # Update credit hours with the value from the dictionary

    return best_match, credit_hours  # Return two values

In [73]:
def main_function(student_detail):
    student_detail['courseName'] = student_detail['courseName'].str.replace(' Lab', '')
    df_grouped = student_detail.groupby('courseName')['points'].sum().reset_index()
    studied_courses = df_grouped.set_index('courseName')['points'].to_dict()
    all_courses = [core_courses, general_education_courses, university_elective_courses, math_science_foundation_courses, domain_courses, cs_elective_courses, support_courses]
    
    total_credit_hours = 0
    missing_or_failed_courses = []

    for course_list in all_courses:
        for course, hours in course_list.items():
            theory_hours, lab_hours = map(int, hours.split('-')) if '-' in hours else (int(hours), 0)
            total_hours = theory_hours + lab_hours

            # Find closest match and update course and credit hours
            closest_match, credit_hours = find_closest_match(course)
            if course != closest_match:
                student_detail['courseName'] = closest_match
                course = closest_match

            if course not in studied_courses:
                missing_or_failed_courses.append(course)
                continue

            # Update credit hours
            total_credit_hours += studied_courses[course]

            if studied_courses[course] < 1 or studied_courses[course] < total_hours:
                missing_or_failed_courses.append(course)

    if course_list == support_courses:
        if len(course_list) - len(missing_or_failed_courses) < 3:
            return f"The degree cannot be completed as the following support courses have not been studied, not passed, or failed: {', '.join(missing_or_failed_courses)}. Total credit hours covered: {total_credit_hours}"
    elif missing_or_failed_courses:
        return f"The degree cannot be completed as the following courses have not been studied, not passed, or failed: {', '.join(missing_or_failed_courses)}. Total credit hours covered: {total_credit_hours}"

    return f"All required courses have been studied and passed. Total credit hours covered: {total_credit_hours}"


In [74]:
df=pd.read_csv('Test Std-0101.csv')
main_function(df)
    


C:\Users\ayesh\AppData\Local\Temp\ipykernel_10752\238021441.py:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = nlp(course_name).similarity(nlp(key))


'The degree cannot be completed as the following support courses have not been studied, not passed, or failed: Data Structures & Algorithms, Discrete Structures, Software Engineering, Information Security, Technical & Business Writing, Communication & Presentation Skills, Professional Practices, Introduction to Info. & Comm. Technologies, Islamic Studies/ Ethics, Foreign Language, Social Service, Calculus & Analytical Geometry, Linear Algebra, Design & Analysis of Algorithms, Parallel & Distributed Computing, Artificial Intelligence, Theory of Automata, CS Elective – 1, CS Elective – 2, CS Elective – 3, CS Elective – 4, CS Elective – 5, Multi-variate Calculus, Graph Theory, Theory of Programming Languages, Numerical Computing. Total credit hours covered: 101.02'